In [14]:
from model import Resnet
level_id = 4

In [15]:
# from utils import split_data
# import torch
# test_cifar_data = torch.ones(2, 3, 32, 32)
# x_a, x_b = split_data(test_cifar_data, 'cifar10')
# print(x_a.shape)

In [16]:
# test_data = torch.ones(2, 3, 64, 64)
# client, server = Resnet(level=level_id)
# output = client(test_data)
# print(output.shape)


In [17]:
# decoder_input = output
# print(decoder_input.shape)

In [18]:
# from model import cifar_discriminator_model, resnet_discriminator
# d = resnet_discriminator(decoder_input.shape[1], level=level_id)

In [19]:
# d_output = d(output)
# print(d_output.shape)

In [20]:
# from model import resent_decoder
# d_input_shape = output.shape[1:]
# decoder = resent_decoder(d_input_shape, level=level_id)
# decoder_output = decoder(decoder_input)
# print(decoder_output.shape)

In [21]:
#########分界线#######################

In [22]:
from model import Resnet
import torch
import torchvision
import torchvision.transforms as transforms
import copy
from vfl import VFLNN, Client, Server
from utils import split_data
import torch.nn.functional as F


In [23]:
tiny_normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

In [24]:
train_dataset = torchvision.datasets.ImageFolder(root='./data/tiny-imagenet-200/train', 
                                                         transform=transforms.Compose([transforms.ToTensor(),
                                                          tiny_normalize])
                                                         )
test_dataset = torchvision.datasets.ImageFolder(root='./data/tiny-imagenet-200/val', 
                                                        transform= transforms.Compose([transforms.ToTensor(),
                                                            tiny_normalize])
                                                        )

In [25]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

In [26]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [28]:
bottom_model1, top_model = Resnet(level=2)
bottom_model2 = copy.deepcopy(bottom_model1)
bottom_model1, bottom_model2, top_model = bottom_model1.to(device), bottom_model2.to(device), top_model.to(device)
client1 = Client(bottom_model1)
client2 = Client(bottom_model2)
server = Server(top_model, 3)


In [29]:
client_optimizer = torch.optim.Adam(bottom_model1.parameters(), lr=0.001)
client2_optimizer = torch.optim.Adam(bottom_model2.parameters(), lr=0.001)
server_optimizer = torch.optim.Adam(top_model.parameters(), lr=0.001)
client_optimizer = [client_optimizer, client2_optimizer]
tar_vflnn = VFLNN(client1, client2, server, client_optimizer, server_optimizer)
for epoch in range(90):
    print("Epoch: ", epoch)
    tar_vflnn.train()
    train_loss = 0
    for i , (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        tar_vflnn.zero_grads()
        x_a, x_b = split_data(data, 'tinyImagnet')

        vfl_ouput = tar_vflnn(x_a, x_b)
        vfl_loss = F.cross_entropy(vfl_ouput, target)
        vfl_loss.backward()

        tar_vflnn.backward()

        tar_vflnn.step()
        train_loss += vfl_loss.item() * data.size(0)
    train_loss = train_loss / len(train_loader.dataset)
    print("Train Loss: ", train_loss)

    tar_vflnn.eval()

    print("----------------------------Test-----------------------------")
    correct = 0
    total = 0
    with torch.no_grad():
        for test_data, test_target in test_loader:
            test_data, test_target = test_data.to(device), test_target.to(device)
            x_a, x_b = split_data(test_data, 'tinyImagnet')
            outputs = tar_vflnn(x_a, x_b)
            _, predicted = torch.max(outputs.data, 1)
            total += test_target.size(0)
            correct += (predicted == test_target).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the network on the test images: {accuracy:.2f}%')

Epoch:  0


KeyboardInterrupt: 